In [2]:
!pip install bs4
!pip install hanja

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=8df6f0362a3b5c32573d4b499e006c4382bbc7ccd2172d960b003c0480a08cbb
  Stored in directory: /home/work/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 120 kB 13.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 265 kB 26.4 MB/s            
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hanja: filename=hanja-0.13.3-py3-none-any.whl size=128390 sha256=61524edec697ac942fc2948d7e8cf33b8aa2e407ae4ef818703f003e701a52de
  Stored in directo

In [3]:
from bs4 import BeautifulSoup
import requests
import time
import hanja
import re
import json
from datetime import datetime, timedelta

In [6]:
def clean_text(new):
    new = new.replace('美', '미국')
    new = new.replace('韓', '한국')
    new = new.replace('中', '중국')
    new = new.replace('日', '일본')
    new = new.replace('英', '영국')
    new = new.replace('獨', '독일')
    new = new.replace('强', '강세')
    new = new.replace('↑', ' 상승')
    new = new.replace('↓', ' 하락')
    new = new.replace('-', '감소')
    new = new.replace('&amp;', '&')
    new = re.sub('\[.*^\]\]', '', new)
    new = re.sub("[-=+,#/\?:^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·◆]"," ",new)
    new = ' '.join(new.split())
    new = hanja.translate(new, 'substitution')
    return new

In [11]:
def crawling_news_all(start, last):

  start_date = datetime.strptime(start, '%Y-%m-%d')
  last_date = datetime.strptime(last, '%Y-%m-%d')

  data = {}
  split_data = {}
  new_string_split = []

  while start_date <= last_date:
      dates = start_date.strftime('%Y-%m-%d')
      print(dates)
      url = f"https://finance.naver.com/news/mainnews.naver"
      req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'}, params={'date': dates})
      html = BeautifulSoup(req.text, 'lxml')
      news = html.find_all('dd', class_="articleSubject")
      news_title=[]
      news_title_split=[]
      for new in news:
          new = new.find('a')
          news_href = "https://finance.naver.com" + new.attrs['href']
          new_string = ""

          # subtracting news summary or first paragraph
          req = requests.get(news_href, headers={'User-agent': 'Mozilla/5.0'})
          html = BeautifulSoup(req.text, 'lxml')
          
          article = html.find(class_="articleCont")
          if article is not None:
            #print('hello', article.find('div'))
            if article.find('div', {'class' : 'link_news'}) is not None:
                article.find('div', {"class" : 'link_news'}).decompose()
            article = clean_text(article.text)
            #print(article)
            new_string = article
            new_string_split = str(article).split('.')
          

          # subtracting news title
          new_string += new.string
          new_string_split.append(new.string)
          new_string = clean_text(new_string)
          for sent in new_string_split:
            news_title_split.append(clean_text(sent))
          news_title.append(new_string)
    
      data[dates] = news_title
      split_data[dates] = news_title_split
    
      time.sleep(0.01)
      start_date += timedelta(days=1)

  return data, split_data

START_DATE = '2012-01-01'
END_DATE = '2022-12-31'

data, split_data = crawling_news_all(START_DATE, END_DATE)

with open(START_DATE + '_' + END_DATE + ".json", 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False)

2012-01-01
2012-01-02
2012-01-03
2012-01-04
2012-01-05
2012-01-06
2012-01-07
2012-01-08
2012-01-09
2012-01-10
2012-01-11
2012-01-12
2012-01-13
2012-01-14
2012-01-15
2012-01-16
2012-01-17
2012-01-18
2012-01-19
2012-01-20
2012-01-21
2012-01-22
2012-01-23
2012-01-24
2012-01-25
2012-01-26
2012-01-27
2012-01-28
2012-01-29
2012-01-30
2012-01-31
2012-02-01
2012-02-02
2012-02-03
2012-02-04
2012-02-05
2012-02-06
2012-02-07
2012-02-08
2012-02-09
2012-02-10
2012-02-11
2012-02-12
2012-02-13
2012-02-14
2012-02-15
2012-02-16
2012-02-17
2012-02-18
2012-02-19
2012-02-20
2012-02-21
2012-02-22
2012-02-23
2012-02-24
2012-02-25
2012-02-26
2012-02-27
2012-02-28
2012-02-29
2012-03-01
2012-03-02
2012-03-03
2012-03-04
2012-03-05
2012-03-06
2012-03-07
2012-03-08
2012-03-09
2012-03-10
2012-03-11
2012-03-12
2012-03-13
2012-03-14
2012-03-15
2012-03-16
2012-03-17
2012-03-18
2012-03-19
2012-03-20
2012-03-21
2012-03-22
2012-03-23
2012-03-24
2012-03-25
2012-03-26
2012-03-27
2012-03-28
2012-03-29
2012-03-30
2012-03-31

2014-01-15
2014-01-16
2014-01-17
2014-01-18
2014-01-19
2014-01-20
2014-01-21
2014-01-22
2014-01-23
2014-01-24
2014-01-25
2014-01-26
2014-01-27
2014-01-28
2014-01-29
2014-01-30
2014-01-31
2014-02-01
2014-02-02
2014-02-03
2014-02-04
2014-02-05
2014-02-06
2014-02-07
2014-02-08
2014-02-09
2014-02-10
2014-02-11
2014-02-12
2014-02-13
2014-02-14
2014-02-15
2014-02-16
2014-02-17
2014-02-18
2014-02-19
2014-02-20
2014-02-21
2014-02-22
2014-02-23
2014-02-24
2014-02-25
2014-02-26
2014-02-27
2014-02-28
2014-03-01
2014-03-02
2014-03-03
2014-03-04
2014-03-05
2014-03-06
2014-03-07
2014-03-08
2014-03-09
2014-03-10
2014-03-11
2014-03-12
2014-03-13
2014-03-14
2014-03-15
2014-03-16
2014-03-17
2014-03-18
2014-03-19
2014-03-20
2014-03-21
2014-03-22
2014-03-23
2014-03-24
2014-03-25
2014-03-26
2014-03-27
2014-03-28
2014-03-29
2014-03-30
2014-03-31
2014-04-01
2014-04-02
2014-04-03
2014-04-04
2014-04-05
2014-04-06
2014-04-07
2014-04-08
2014-04-09
2014-04-10
2014-04-11
2014-04-12
2014-04-13
2014-04-14
2014-04-15

2016-01-30
2016-01-31
2016-02-01
2016-02-02
2016-02-03
2016-02-04
2016-02-05
2016-02-06
2016-02-07
2016-02-08
2016-02-09
2016-02-10
2016-02-11
2016-02-12
2016-02-13
2016-02-14
2016-02-15
2016-02-16
2016-02-17
2016-02-18
2016-02-19
2016-02-20
2016-02-21
2016-02-22
2016-02-23
2016-02-24
2016-02-25
2016-02-26
2016-02-27
2016-02-28
2016-02-29
2016-03-01
2016-03-02
2016-03-03
2016-03-04
2016-03-05
2016-03-06
2016-03-07
2016-03-08
2016-03-09
2016-03-10
2016-03-11
2016-03-12
2016-03-13
2016-03-14
2016-03-15
2016-03-16
2016-03-17
2016-03-18
2016-03-19
2016-03-20
2016-03-21
2016-03-22
2016-03-23
2016-03-24
2016-03-25
2016-03-26
2016-03-27
2016-03-28
2016-03-29
2016-03-30
2016-03-31
2016-04-01
2016-04-02
2016-04-03
2016-04-04
2016-04-05
2016-04-06
2016-04-07
2016-04-08
2016-04-09
2016-04-10
2016-04-11
2016-04-12
2016-04-13
2016-04-14
2016-04-15
2016-04-16
2016-04-17
2016-04-18
2016-04-19
2016-04-20
2016-04-21
2016-04-22
2016-04-23
2016-04-24
2016-04-25
2016-04-26
2016-04-27
2016-04-28
2016-04-29

2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-17
2018-02-18
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-03
2018-03-04
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-10
2018-03-11
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-17
2018-03-18
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-24
2018-03-25
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-07
2018-04-08
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-14
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-21
2018-04-22
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-28
2018-04-29
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-05
2018-05-06
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-13
2018-05-14

2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28

2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-02
2022-04-03
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-09
2022-04-10
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12

In [10]:
data

{'2022-11-01': ['9월 구인건수 전월 대비 늘어나추가적인 긴축 필요성 해석 제기2일 FOMC서 12월 발언 내용 중요뉴욕증권거래소 NYSE 내부. AFP연합뉴스 서울경제 11월 연방공개시장위원회 FOMC 결과 발표를 하루 앞두고 미국 노동시장의 강함이 다시 한번 입증되면서 뉴욕증시의 주요 지수가 하락 출발했다.1일 현지 시간 미 경제 방송 CNBC에 따르면 이날 오전10시14분 현재 뉴욕증권거래소 NYSE 의 다우존스산업평균지수가 전날보다 74.56포인트 감소0.23% 내린 3만2861.80에 거래되고 있다.스탠더드앤드푸어스 S&P 500이 6.70포인트 감소0.17% 하락한 3865.44 나스닥은 32.90포인트 감소0.30% 떨어진 1만955.24를 기록 중이다.전날 연 4.09%까지 갔던 10년 만기 미 국채금리가 이날 오전 한때 3.92%대까지 내려왔다. 11월 FOMC가 이날부터 시작한 가운데 2일 있을 결과 발표에서 12월 기준금리 인상속도 조절에 관한 기대감이 널리 퍼졌기 때문이다. CME 페드워치에 따르면 1일 오전9시57분 현재 11월 0.75%p 인상 확률은 89.4% 12월 0.5%p는 51.5%다. CNBC는 11월 FOMC에서 0.75%포인트의 금리인상이 이뤄질 것이라고 많은 이들이 예상하고 있다 며 투자자들의 관심은 긴축속도가 느려지는 징후가 있는지이며 이를 위해 성명서와 제롬 파월 의장의 기자회견을 눈여겨 볼 것 이라고 설명했다.하지만 강력한 고용 관련 지표가 발목을 잡고 있다. 이날 나온 9월 구인이직 보고서를 보면 9월 구인 건수가 1071만7000건으로 8월 1028만 건 보다 되레 증가했다. 감소하던 것이 다시 늘어난 것이다. 그만큼 노동시장이 강하며 이는 추가적인 긴축을 의미할 수 있다는 게 시장의 해석이다. 12월에 속도조절이 가능하겠느냐에 대한 회의감도 고개를 들고 있다.종목별로는 우버가 예상보다 큰 손실폭을 기록했지만 매출이 전년 대비 72% 폭등 주가가 15% 넘게 오르고 있다. 예상을 깬 실적을 올린 화